# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [4]:
#pip install --upgrade langchain

In [5]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [6]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [7]:
file = 'dataset.csv'
loader = CSVLoader(file_path=file)

In [8]:
from langchain.indexes import VectorstoreIndexCreator

In [9]:
#pip install docarray

In [10]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

AuthenticationError: Incorrect API key provided: sk-qBW8I***************************************n8Hu. You can find your API key at https://platform.openai.com/account/api-keys.